## 05 Auslese aller Zürcher Webseiten von Homegate

Nun wollen wir alle Seiten runterladen, nicht nur nur eine

In [33]:
#Unsere Imports
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [34]:
#URL Vorbereitung
base_url = 'https://www.homegate.ch/mieten/immobilien/plz-8000/trefferliste?loc=geo-zipcode-8000&ep='

#Hier bereite ich vor, wieviele Seiten besucht werden müssen
response = requests.get(base_url + '1')
soup = BeautifulSoup(response.content, 'html.parser')
max_page = int(soup.find_all(class_='router-link-active')[-3].text[-2:])

In [35]:
all_prices = []
all_rooms_livingspace_details = []
all_addresses = []

for page in range(1, max_page + 1):
    response = requests.get(base_url + str(page))
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Für die Preise (Aufpassen, diese Nummern am Ende deuten darauf hin, dass die gelegentlich ändern)
    prices = [price.text.strip() for price in soup.find_all(class_="HgListingCard_price_sIIoV")]
    all_prices.extend(prices)
    
    # Wohnraum und Zimmer (Aufpassen, diese Nummern am Ende deuten darauf hin, dass die gelegentlich ändern)
    rooms_livingspace_details = [detail.text.strip() for detail in soup.find_all(class_='HgListingRoomsLivingSpace_roomsLivingSpace_FiW9E')]
    all_rooms_livingspace_details.extend(rooms_livingspace_details)
    
    # Adresse
    addresses = [item.text.strip() for item in soup.find_all(attrs={"translate": "no"})]
    all_addresses.extend(addresses)

# Create DataFrame and save to CSV
df = pd.DataFrame({
    'Price': all_prices,
    'Rooms & Living Space Details': all_rooms_livingspace_details,
    'Address': all_addresses
})

df.to_csv('scraped_data.csv', index=False)

print(f"Data saved to scraped_data.csv")


Data saved to scraped_data.csv


Jetzt bringen wir das bei Pandas in einer Form, damit wir damit ausrechnen können, wie teuer ein Quadratmeter aktuell in Zürich ist. Unter der PLZ 8001. Wir können das auch nach Strasse herunterrechnen. 